In [1]:
#import boto3, json
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
columns_=['Siren','Type','Adresse_Ligne1','Adresse_Ligne2','Adresse_Ligne3',
          #'Code_Postal',
          'Ville','Code_Commune','ID_Etablissement','Date_Greffe','Libelle_Evt']
columns_add = columns_ + [ 'file_timestamp', 'Libelle_Evt2']
dtype_={'Siren': 'object',
        'Type': 'object',
        'Adresse_Ligne1': 'object',
        'Adresse_Ligne2': 'object',
        'Adresse_Ligne3': 'object',
        'Code_Postal': 'object',
        'Ville': 'object',
        'Code_Commune': 'object',
        'ID_Etablissement': 'object',
        'Date_Greffe': 'object',
        'Libelle_Evt': 'object'
       }

dtype_2={'Siren': 'object',
        'Type': 'object',
        'Adresse_Ligne1': 'object',
        'Adresse_Ligne2': 'object',
        'Adresse_Ligne3': 'object',
        'Code_Postal': 'object',
        'Ville': 'object',
        'Code_Commune': 'object',
        'ID_Etablissement': 'object',
        'Date_Greffe': 'object',
        'Libelle_Evt': 'object',
        'Code Greffe': 'object',
        'Domiciliataire_Siren': 'object',
       }

#data_evt.dtypes
#data_evt_enriched.dtypes

In [ ]:
data_initial= dd.read_csv("temp/initial_ETS.gz", 
                          #usecols =["Siren"],
                          dtype={'Adresse_Ligne1': 'object',
                                 'Code_Commune': 'object',
                                 'Code_Postal': 'object',
                                 'Domiciliataire_Greffe': 'object'},
                          compression='gzip',
                          blocksize=None,
                          low_memory=False
                      )

In [ ]:
data_initial.compute().shape

In [ ]:
'''data_evt= dd.read_csv("temp/2017_EVT_ETS.gz", 
                      dtype=dtype_2,
                      compression='gzip',
                      blocksize=None,
                      low_memory=False
                      )
#data_evt.head()
data_evt.compute().shape
'''

data_evt= pd.read_csv("temp/2017_EVT_ETS.gz", 
                      dtype=dtype_2,
                      compression='gzip',
                      low_memory=False
                      )
data_evt.shape

In [ ]:
#contient 2017,2018,2019
'''data_evt_enriched= dd.read_csv("temp/ETS_enriched.gz", 
                      dtype=dtype_,
                      compression='gzip',
                      blocksize=None,
                      low_memory=False,
                      usecols=columns_add
                      )
data_evt_enriched.compute().drop_duplicates(keep='first')
data_evt_enriched.compute().shape
'''

data_evt_enriched= pd.read_csv("temp/ETS_enriched.gz", 
                      dtype=dtype_,
                      compression='gzip',
                      low_memory=False,
                      usecols=columns_add
                      )
data_evt_enriched.shape

In [ ]:
#Temporaire le temps de refaire tourner l'algo
data_evt_enriched=data_evt_enriched.drop_duplicates()
data_evt_enriched.shape

In [ ]:
#data_evt_enriched.loc[lambda x: x['Siren'].isin(['420407355'])].transpose()

In [ ]:
#data_evt_enriched.dtypes

In [ ]:
data_evt_enriched['file_timestamp']=pd.to_datetime(data_evt_enriched['file_timestamp'])

In [ ]:
data_evt_enriched['year'] = data_evt_enriched['file_timestamp'].dt.year
data_evt_enriched['month'] = data_evt_enriched['file_timestamp'].dt.month

In [ ]:
data_evt_enriched_2017_05 = data_evt_enriched.loc[lambda x: ( x['year'].isin(['2017'])) & 
                                               (x['month'].isin(['5']))]
data_evt_enriched_2017_05.shape

In [3]:
#contient 201705
columns_=['Siren','Type','Adresse_Ligne1','Adresse_Ligne2','Adresse_Ligne3',
          #'Code_Postal',
          'Ville','Code_Commune','ID_Etablissement','Date_Greffe','Libelle_Evt',
          'file_timestamp', 'Libelle_Evt2'
         ]

data_evt_enriched= pd.read_csv("temp/201705_ETS_enriched_Normalized.gz", 
                      dtype=dtype_,
                      compression='gzip',
                      low_memory=False,
                      usecols=columns_add
                      )
data_evt_enriched.shape

ValueError: Usecols do not match columns, columns expected but not found: ['Siren']

In [ ]:
result = data_evt.merge(data_evt_enriched)
result.shape

In [ ]:
result = pd.concat([data_evt,data_evt_enriched],sort=True,ignore_index=True)
result.shape

In [ ]:
result = pd.merge(data_evt,
                  data_evt_enriched,
                  on=columns_,
                  how='left', 
                 indicator=True
                 )
result.shape

In [ ]:
#data_evt.compute().loc[lambda x: x['Siren'].isin(['420407355'])].transpose()

In [ ]:
#test = data_evt_enriched.compute().loc[lambda x: x['Siren'].isin(['420407355'])].drop_duplicates(keep='first').transpose()
#test

In [ ]:
result.loc[lambda x: x['Siren'].isin(['420407355'])].transpose()

In [ ]:
data_evt=result

In [ ]:
ex1='420407355'
data_evt_ex1 = data_evt.compute().loc[lambda x: x['Siren'].isin([ex1])]
data_initial_ex1 = data_initial.compute().loc[lambda x: x['Siren'].isin([ex1])]
merge_ex1 = pd.concat([data_evt_ex1, data_initial_ex1], axis=0)
merge_ex1.transpose()

In [ ]:
# Count SIREN by Nb Evts (50% of 1 Evt)
(data_evt
 .groupby('Siren')['Siren']
 .count()
 .rename('count')
 .reset_index()
 .groupby('count')['count']
 .count()
 #.compute()
)

In [ ]:
data_1evt_idx = (data_evt
 .groupby('Siren')['Siren']
 .count()
 .rename('evt_count')
 .reset_index()
 .compute()
)

In [ ]:
data_1evt = data_evt.merge(data_1evt_idx).compute().loc[lambda x: x['evt_count'].isin(['1'])]
data_1evt

In [ ]:
data_1evt = data_evt.merge(data_1evt_idx).loc[data_evt['evt_count'].isin(['1'])].compute()


In [ ]:
#data_initial_1evt = data_initial.merge(data_1evt_idx).compute().loc[lambda x: x['evt_count'].isin(['1'])]
#data_initial_1evt

In [ ]:
ex7='344734942'
data_evt_ex7 = data_1evt.loc[lambda x: x['Siren'].isin([ex7])]
data_initial_ex7 = data_initial_1evt.loc[lambda x: x['Siren'].isin([ex7])]
merge_ex7 = pd.concat([data_evt_ex7, data_initial_ex7], axis=0)
merge_ex7.transpose()

In [ ]:
# Count SIREN by Nb Init (50% of 1 Init)
(data_initial
 .groupby('Siren')['Siren']
 .count()
 .rename('init_count')
 .reset_index()
 .groupby('init_count')['init_count']
 .count()
 .compute()
)

In [ ]:
data_1init_idx = (data_initial
 .groupby('Siren')['Siren']
 .count()
 .rename('init_count')
 .reset_index()
 .compute()
)

In [ ]:
data_1init = data_initial.merge(data_1init_idx).compute().loc[lambda x: x['init_count'].isin(['1'])]
data_1init

In [ ]:
merge_ = pd.concat([data_1evt, data_1init], axis=0)
merge_.head()

In [ ]:
len(merge_)

In [ ]:
merge_.loc[lambda x: x['Siren'].isin(['823358841'])].transpose()

In [ ]:
for i,row in data_1evt.loc[data_1evt['Type_Exploitation'].isnull(),:].iterrows():
    #print(row['Siren'])
    ind = tuple([row['Siren'],row['Type'],row['Adresse_Ligne1'],row['Adresse_Ligne2'],row['Adresse_Ligne3'],row['ID_Etablissement']])
    print(ind)
    #data_1init.loc[i,'Type_Exploitation'] = impute_grps.loc[ind].values[0]

In [ ]:
#MERGE ET VERIF UNICITE

In [ ]:
# TEST 1 on=['Siren','ID_Etablissement']
test=data_initial.merge(data_evt,on=['Siren','ID_Etablissement'],how='right',indicator=True).compute()
test.groupby('_merge')['_merge'].count()

In [ ]:
test.loc[test['_merge'].isin(['both'])].head(10).to_excel('test.xlsx')

In [ ]:
# on=['Siren','ID_Etablissement'] => n'est pas suffisant pour avoir des établissements uniques (ex de 572086577)

In [ ]:
# TEST 2 on=['Siren','ID_Etablissement','Type']
test=data_initial.merge(data_evt,on=['Siren','ID_Etablissement','Type'],how='right',indicator=True).compute()
test.groupby('_merge')['_merge'].count()

In [ ]:
test.loc[test['_merge'].isin(['both'])].head(10).to_excel('test2.xlsx')

In [ ]:
# on=['Siren','ID_Etablissement','Type'] => n'est pas suffisant pour avoir des établissements uniques (ex de 572086577)

In [ ]:
# TEST 3 on=['Siren','ID_Etablissement','Type','Ville']
test=data_initial.merge(data_evt,on=['Siren','ID_Etablissement','Type','Ville'],how='right',indicator=True).compute()
test.groupby('_merge')['_merge'].count()

In [ ]:
test.loc[test['_merge'].isin(['both'])].head(100).to_excel('test3.xlsx')